# PIPEX

## Makefile

In [21]:
%%file ../pipex_working/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2023/01/30 14:27:11 by dpentlan          #+#    #+#              #
#    Updated: 2023/01/30 14:27:14 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

NAME = pipex

# ***** Sources *****

SRCS = pipex.c \

SRCS_DIR = srcs/

# ***** Includes *****

INCS = pipex.h

INCS_DIR = includes/

# ***** Objects *****

OBJS = $(patsubst %.c,$(OBJS_DIR)%.o,$(SRCS))

OBJS_DIR = objs/

# ***** Compile and Archiver *****

CFLAGS = -Wall -Wextra -Werror -g

CC = cc

LIBFT = -Llibft -lft

# ***** Build *****

all: $(NAME)

bonus: $(BONUS)

$(NAME): $(OBJS) $(OBJS_MAN)
	make -C libft
	$(CC) $(CFLAGS) $(OBJS) $(LIBFT) -o $(NAME)

$(OBJS_DIR)%.o: $(SRCS_DIR)%.c
	@mkdir -p $(OBJS_DIR)
	$(CC) -c $(CFLAGS) -I$(INCS_DIR) $< -o $@

clean:
	@make -C libft clean --silent
	@rm -f $(OBJS)
	@rm -rf $(OBJS_DIR)

fclean:	clean
	@make -C libft fclean --silent
	@rm -f $(NAME)
	@rm -f gdb.txt

re:	fclean all


Overwriting ../pipex_working/Makefile


## HEADERS

In [45]:
%%file ../pipex_working/includes/pipex.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.h                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:57:04 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/20 12:57:37 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef PIPEX_H
# define PIPEX_H

# include "../libft/includes/libft.h"
# include <unistd.h>
# include <sys/wait.h>

#endif


Overwriting ../pipex_working/includes/pipex.h


## SOURCES

In [1]:
%%file ../pipex_working/srcs/pipex.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.c                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:56:25 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/20 12:56:37 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	main(int argc, char **argv, char **envp)
{
	pid_t	pid;
	int		ret;

	if (argc) {}
	if (argv) {}
	if (envp) {}

	ret = 1;
	pid = fork();
	ft_printf("pid = %d\n", pid);

	if (pid != 0)
	{
		ft_printf("waiting: %d\n", pid);
		wait(NULL);
		if (ret < 0)
			return (1);
		ft_printf("waiting over: %d\n", pid);
	}
	
	if (pid == 0)
		ft_printf("Hey! I'm the child! %d\n", pid);
	if (pid != 0)
		ft_printf("Hey! I'm the parent! %d\n", pid);

	return (0);
}


Overwriting ../pipex_working/srcs/pipex.c


## TESTING

### Test Build

In [2]:
%%bash
cd ../pipex_working
make re > /dev/null
./pipex
nm -Du pipex
make fclean

pid = 46320
waiting: 46320
pid = 0
Hey! I'm the child! 0
waiting over: 46320
Hey! I'm the parent! 46320
                 w __cxa_finalize@GLIBC_2.2.5
                 U fork@GLIBC_2.2.5
                 U free@GLIBC_2.2.5
                 w __gmon_start__
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
                 U __libc_start_main@GLIBC_2.34
                 U malloc@GLIBC_2.2.5
                 U __stack_chk_fail@GLIBC_2.4
                 U wait@GLIBC_2.2.5
                 U write@GLIBC_2.2.5


In [2]:
%%bash
# use valgrind with --track-fds=all to see if fds are closing appropriately
valgrind --track-fds=all < file1 cat | wc > file2

==86224== Memcheck, a memory error detector
==86224== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==86224== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==86224== Command: cat
==86224== 
==86224== 
==86224== FILE DESCRIPTORS: 0 open (0 std) at exit.
==86224== 
==86224== HEAP SUMMARY:
==86224==     in use at exit: 0 bytes in 0 blocks
==86224==   total heap usage: 1,051 allocs, 1,051 frees, 205,320 bytes allocated
==86224== 
==86224== All heap blocks were freed -- no leaks are possible
==86224== 
==86224== For lists of detected and suppressed errors, rerun with: -s
==86224== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)
